In [ ]:
import json
import re

from bs4 import BeautifulSoup
import string
from string import digits
from datetime import datetime

from gensim.models import CoherenceModel
from gensim.models import Phrases
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from spacy.lang.en import STOP_WORDS

from gensim import corpora, models
import pyLDAvis.gensim
import numpy as np

try:
  from sbt_common import SbtGlobalCommon
  _LOGGER = SbtGlobalCommon.get_global_logger().info
except:
  _LOGGER = print


def timeit(f):
  def timed(*args, **kw):
    tic = datetime.now()
    result = f(*args, **kw)
    tac = datetime.now()
    _LOGGER(" {}: DONE IN {}".format(f.__name__, tac - tic))
    return result
  return timed


class BasicProcessing(object):

  @timeit
  def __init__(self, read_local_file=False):
    # local storage file
    self._data_file = './processed_data.npy'
    # read from previously processed local file
    # _DATA = json.loads(open(self._data_file, 'r').read()) \
    #   if read_local_file else None
    _DATA = np.load(self._data_file).item() if read_local_file else None
    # set loaded from local file flag to T or F
    self._loaded_from_file = True if _DATA else False

    # create stopwords list
    _STOPWORDS_COLLECTION = stopwords.words('english')
    _STOPWORDS_COLLECTION.extend(list(ENGLISH_STOP_WORDS))
    _STOPWORDS_COLLECTION.extend(list(STOP_WORDS))
    self._stopwords = set(_STOPWORDS_COLLECTION)
    # additional stopwords
    additional_stopwords = ["say", "year", "week"]
    [self._stopwords.add(new_stopword)
     for new_stopword in additional_stopwords] \
      if len(additional_stopwords) > 0 else None

    # english contractions list
    self._english_contractions = {
      "ain't": "am not",
      "aren't": "are not",
      "can't": "cannot",
      "can't've": "cannot have",
      "'cause": "because",
      "could've": "could have",
      "couldn't": "could not",
      "couldn't've": "could not have",
      "didn't": "did not",
      "doesn't": "does not",
      "don't": "do not",
      "hadn't": "had not",
      "hadn't've": "had not have",
      "hasn't": "has not",
      "haven't": "have not",
      "he'd": "he would",
      "he'd've": "he would have",
      "he'll": "he will",
      "he'll've": "he will have",
      "he's": "he is",
      "how'd": "how did",
      "how'd'y": "how do you",
      "how'll": "how will",
      "how's": "how is",
      "i'd": "I would",
      "i'd've": "I would have",
      "i'll": "I will",
      "i'll've": "I will have",
      "i'm": "I am",
      "i've": "I have",
      "isn't": "is not",
      "it'd": "it would",
      "it'd've": "it would have",
      "it'll": "it will",
      "it'll've": "it will have",
      "it's": "it is",
      "let's": "let us",
      "ma'am": "madam",
      "mayn't": "may not",
      "might've": "might have",
      "mightn't": "might not",
      "mightn't've": "might not have",
      "must've": "must have",
      "mustn't": "must not",
      "mustn't've": "must not have",
      "needn't": "need not",
      "needn't've": "need not have",
      "o'clock": "of the clock",
      "oughtn't": "ought not",
      "oughtn't've": "ought not have",
      "shan't": "shall not",
      "sha'n't": "shall not",
      "shan't've": "shall not have",
      "she'd": "she would",
      "she'd've": "she would have",
      "she'll": "she will",
      "she'll've": "she will have",
      "she's": "she is",
      "should've": "should have",
      "shouldn't": "should not",
      "shouldn't've": "should not have",
      "so've": "so have",
      "so's": "so is",
      "that'd": "that would",
      "that'd've": "that would have",
      "that's": "that is",
      "there'd": "there would",
      "there'd've": "there would have",
      "there's": "there is",
      "they'd": "they would",
      "they'd've": "they would have",
      "they'll": "they will",
      "they'll've": "they will have",
      "they're": "they are",
      "they've": "they have",
      "to've": "to have",
      "wasn't": "was not",
      "we'd": "we would",
      "we'd've": "we would have",
      "we'll": "we will",
      "we'll've": "we will have",
      "we're": "we are",
      "we've": "we have",
      "weren't": "were not",
      "what'll": "what will",
      "what'll've": "what will have",
      "what're": "what are",
      "what's": "what is",
      "what've": "what have",
      "when's": "when is",
      "when've": "when have",
      "where'd": "where did",
      "where's": "where is",
      "where've": "where have",
      "who'll": "who will",
      "who'll've": "who will have",
      "who's": "who is",
      "who've": "who have",
      "why's": "why is",
      "why've": "why have",
      "will've": "will have",
      "won't": "will not",
      "won't've": "will not have",
      "would've": "would have",
      "wouldn't": "would not",
      "wouldn't've": "would not have",
      "y'all": "you all",
      "y'all'd": "you all would",
      "y'all'd've": "you all would have",
      "y'all're": "you all are",
      "y'all've": "you all have",
      "you'd": "you would",
      "you'd've": "you would have",
      "you'll": "you will",
      "you'll've": "you will have",
      "you're": "you are",
      "you've": "you have"
    }

    # create punctuation list
    self._punctuation = list(string.punctuation)
    # # - do not remove ' (keep possessive forms)
    # self._punctuation.remove("'")

    self.wordpress_id = _DATA['wordpressId'] if _DATA else []
    self.created_at = _DATA['createdAt'] if _DATA else []
    self.modified_at = _DATA['modifiedAt'] if _DATA else []
    self.raw_content = _DATA['raw_content'] if _DATA else []
    self.clean_content = _DATA['clean_content'] if _DATA else []
    self.tokens = _DATA['tokens'] if _DATA else []
    self.normalized_tokens = _DATA['normalized_tokens'] if _DATA else []
    self.bigram_tokens = _DATA['bigram_tokens'] if _DATA else []

  @timeit
  def _save_data(self):
    data_dict = {'raw_content': self.raw_content,
                 'clean_content': self.clean_content,
                 'tokens': self.tokens,
                 'normalized_tokens': self.normalized_tokens,
                 'wordpress_id': self.wordpress_id,
                 'created_at': self.created_at,
                 'modified_at': self.modified_at}
    np.save(self._data_file, data_dict)
    # with open(self._data_file, 'w') as f:
    #   json.dump(data_dict, f)

  @timeit
  def _set_content(self, full_path_to_file):
    # reading the json file as plain text
    with open(full_path_to_file, 'r') as f:
      content = f.read()
    # converting each json object into a dict
    data = [json.loads(x) for x in content.splitlines()]
    # extracting the content
    self.raw_content = [x['contentText'] for x in data
                        if x['type'].lower() == 'post']
    self.wordpress_id = [x['wordpressId'] for x in data
                         if x['type'].lower() == 'post']
    self.created_at = [x['createdAt'] for x in data
                       if x['type'].lower() == 'post']
    self.modified_at = [x['modifiedAt'] for x in data
                        if x['type'].lower() == 'post']

    self._save_data() if not self._loaded_from_file else None

  def _set_text(self, text):
    '''
    Test if a text content has been provided.
    If text is None, use self.clean_content.
    If text is None AND self.clean_content is empty, use self.raw_content.
    :param text:
    :return: the appropriate text content.
    '''
    return text if text is not None else self.clean_content \
      if len(self.clean_content) else self.raw_content

  @timeit
  def _remove_punctuation(self, text=None):
    '''
    Remove all punctuation from the text content
    (the punctuation list: self._punctuation).
    :param text:
    :return: text content without punctuation.
    '''
    text = self._set_text(text)
    table = str.maketrans({key: None for key in self._punctuation})
    text = [x.translate(table) for x in text]
    return text

  @timeit
  def _remove_digits(self, text=None):
    '''
    Remove digits from text content.
    :param text:
    :return: text content without digits.
    '''
    text = self._set_text(text)
    # removing digits
    table = str.maketrans('', '', digits)
    text = [x.translate(table) for x in text]
    return text

  @timeit
  def _remove_podcast_header(self, text=None):
    '''
    Remove the Javascript header from Doc's podcast transcripts.
    Regex explanation:
      var\s.*\; -> Matches the word 'var' and everything between spaces and
                     semi-colons that follows 'var'.
      \n+.*\; -> Matches everything between newlines and semi-colons.
    :param text:
    :return: text content without the JS header.
    '''
    text = self._set_text(text)
    text = [re.sub(r"""var\s.*\;\n+.*\;""", " ", content.lower())
            for content in text]
    return text

  @timeit
  def _remove_urls(self, text=None):
    '''
    Remove URLs from text content. The regex pattern was obtained from
    https://stackoverflow.com/a/51884776/5049942 and has the following schema:
      https?:\/\/ -> Matches 'http://' or 'https://'.
      \s* -> Greedily matches any space that follows previous pattern.
      (www\.)? -> Matches any 'www.' that follows previous pattern (if present).
      \s* -> Greedily matches any space that follows previous pattern.
      [\w\S]+ -> Matches any word character or non-whitespaces that follows
                 previous pattern.
    :param text:
    :return: text content without URLs.
    '''
    text = self._set_text(text)
    text = [re.sub(r"""https?:\/\/\s*(www\.)?\s*[\w\S]+""", " ",
                   content.lower()) for content in text]
    return text

  def _remove_html_entities(self, text=None):
    '''
    Remove all ISO 8859-1 (Latin 1) HTML entities.
    (reference: https://cs.stanford.edu/~miles/iso8859.html)
    Regex explanation:
      \&\#?\w+\; -> Matches any word character between '&' and ';'.
    :param text:
    :return: text content without HTML entities.
    '''
    text = self._set_text(text)
    text = [re.sub(r"""\&\#?\w+\;""", "", content) for content in text]
    return text

  @timeit
  def _remove_stopwords(self, text=None):
    '''
    Remove all stopwords from text content
    (the list of stopwords: self._stopwords).
    :param text:
    :return: text content without stopwords.
    '''
    text = self._set_text(text)
    pattern = re.compile(r'\b(' + r'|'.join(self._stopwords) + r')\b\s*')
    text = [pattern.sub("", content.lower()) for content in text]
    return text

  @timeit
  def _expand_contractions(self, text=None):
    '''
    Replace english contractions with their expanded form
    (list of english contractions: self._english_contractions).
    :param text:
    :return:
    '''
    text = self._set_text(text)

    def multiple_replace(text, adict):
      rx = re.compile('|'.join(map(re.escape, adict)))

      def one_xlat(match):
        return adict[match.group(0)]

      return rx.sub(one_xlat, text.lower())

    text = [multiple_replace(content.lower(), self._english_contractions)
            for content in text]

    return text

  @timeit
  def _noise_removal(self):
    '''
    Perform a series of text processing to remove noisy entities.
    :return: text content without noisy entities.
    '''
    text = self._remove_podcast_header()
    text = self._remove_urls(text)
    text = self._remove_html_entities(text)
    text = self._expand_contractions(text)
    text = self._remove_punctuation(text)
    text = self._remove_digits(text)
    text = self._remove_stopwords(text)
    self.clean_content = text
    self._save_data() if not self._loaded_from_file else None

  @timeit
  def _tokenization(self):
    '''
    Break text content into tokens.
    :return: updates self.tokens.
    '''
    tokens = [x.split() for x in self.clean_content]
    self.tokens = tokens
    self._save_data() if not self._loaded_from_file else None

  @timeit
  def _bigrams(self):
    '''
    Break text content into tokens and performs
    text analysis to determine bigrams.
    :return: updates self.bigram_tokens.
    '''
    texts = self.tokens
    bigram = Phrases(texts)
    self.bigram_tokens = [bigram[line] for line in texts]
    self._save_data() if not self._loaded_from_file else None

  @timeit
  def _normalization(self, bigrams=True, stem=False):
    '''
    Perform normalization of text content by lemmatizing or stemming of text.
    :param bigrams: if True, process self.bigram_tokens instead of self.tokens.
    :param stem: if True, perform stemming instead of lemmatization.
    :return: updates self.normalized_tokens.
    '''
    if stem:
      stemmer = PorterStemmer()
      norm_tokens = [[stemmer.stem(t) for t in x] for x in self.bigram_tokens] \
                      if bigrams else \
                    [[stemmer.stem(t) for t in x] for x in self.tokens]
    else:
      lemmer = WordNetLemmatizer()
      norm_tokens = [[lemmer.lemmatize(t.lower(), pos='v') for t in x]
                     for x in self.bigram_tokens] if bigrams else \
                    [[lemmer.lemmatize(t.lower(), pos='v') for t in x]
                     for x in self.tokens]
    self.normalized_tokens = norm_tokens
    self._save_data() if not self._loaded_from_file else None

  @timeit
  def clean(self, path_to_file):
    if not self._loaded_from_file:
      self._set_content(path_to_file)
      self._noise_removal()
      self._tokenization()
      self._bigrams()
      self._normalization()
      # save class
      np.save("sbcontent.npy", self)


class FeatureExtraction(object):

  @timeit
  def __init__(self, content):
    self.content = content
    self.dictionary = corpora.Dictionary(content)
    self.corpus = []
    self.features = []
    self.model = []

    self.bow = None
    self.tfidf = None
    self.lsa = None
    self.lda = None
    self.lsi = None

  @timeit
  def _get_bow(self):
    self.corpus = [self.dictionary.doc2bow(t) for t in self.content]
    self.bow = self.corpus

  @timeit
  def _tfidf(self):
    # create a transformation
    self.tfidf = models.TfidfModel(self.corpus, normalize=True)

  @timeit
  def _lda(self):
    self.lda = models.LdaModel(self.corpus,
                              id2word=self.dictionary,
                              num_topics=10,
                              chunksize=100, decay=0.5,
                              passes=1, update_every=1)

  @timeit
  def _lsa(self):
    pass

  @timeit
  def _lsi(self):
    self.lsi = models.LsiModel(self.corpus,
                               id2word=self.dictionary,
                               num_topics=10,
                               chunksize=100, decay=0.5)

  @timeit
  def extract_features(self, model='lda'):
    self._get_bow()
    self._tfidf()
    if model == 'lsa': self._lsa()
    if model == 'lda': self._lda()
    if model == 'lsi': self._lsi()

  def get_category_items(self, category=None, n_items=None, model='',
                            content='', output=None):
    ### apply the transformation to the corpus
    if model == 'lsa': doc = self.lsa[self.corpus]
    if model == 'lda': doc = self.lda[self.corpus]
    if model == 'lsi': doc = self.lsi[self.corpus]
    if model == '': return None

    ### get the (class, likelihood) for the content
    classifier = [x for x in doc]
    ### sort the results by most likely class
    classifier_sorted = [sorted({y[0]: y[1] for y in x}.items(),
                                key=lambda x: -x[1]) for x in classifier]
    ### get the most likely class and corresponding item index
    most_likely_class = [{i: x[0]} for i, x in enumerate(classifier_sorted)]

    ### selecting data for a given class
    ### 1 - filtering out other classes
    sub1 = [[x for k, v in x.items() if v[0] == category]
            for x in most_likely_class]
    ### 2 - keeping only not empty data
    sub2 = [x[0] for x in sub1 if len(x)]

    ### sort the results
    sub2_sorted = sorted(sub2, key=lambda x: list(x.values()), reverse=True)

    ### get the first n_items of all
    # items = [content[list(x.items())[0][0]] for x in sub2_sorted[:n_items]] \
    #   if n_items else [content[list(x.items())[0][0]] for x in sub2_sorted]
    items = [{k: content[k] for k, v in x.items()} for x in sub2_sorted[:n_items]] \
      if n_items else [{k: content[k] for k, v in x.items()} for x in sub2_sorted]

    ### write the articles to a file
    if output:
      with open(output, 'w') as f:
        json.dump(items, f)
    return items

In [ ]:
path = '/Users/mteodoro/Google Drive/SBCONTENT_ANALYSIS/articles_20180814.json'

basic_process = BasicProcessing()

basic_process.clean(path)

In [ ]:
fe = FeatureExtraction(basic_process.normalized_tokens)

fe.extract_features(model='lda')

## Topic Visualization

In [ ]:
vis = pyLDAvis.gensim.prepare(fe.lda, fe.corpus, fe.dictionary, sort_topics=False)
pyLDAvis.display(vis)

## Topic Selection

In [ ]:
fe.lda.print_topics()

In [ ]:
items = fe.get_category_items(category=3, model='lda',
                       content=basic_process.raw_content,
                       output='natural_resources.json')

In [ ]:
items

In [ ]:
# np.save('model_20180905.npy', fe)

## Model Metrics

In [ ]:
perplexity = fe.lda.log_perplexity(fe.corpus)
print('Perplexity: {}'.format(perplexity))

coherence_model = CoherenceModel(model=fe.lda,
                                   texts=basic_process.normalized_tokens,
                                   dictionary=fe.dictionary,
                                   coherence='c_v')
coherence_lda = coherence_model.get_coherence()
print('Coherence Score: {}'.format(coherence_lda))